<h1>Задание 4. Метод сеток для решения уравнения параболического типа</h1>
<h2 align = "center"><font color="#6f8600">Вариант 8</font></h2>
<p>Будем искать решение следующей задачи:</p>
$\frac{\partial u}{\partial t} = \frac{\partial}{\partial x}((x+1)\frac{\partial u}{\partial x}) - u + f(x, t)$, где<br/>
$u(x, 0) = \varphi(x), \ \ 0 \leqslant x \leqslant 1$<br/>
$u(0, t) = \alpha(t), \ \ u(1, t) + \frac{\partial u}{\partial x}\bigg|_{x=1}= \beta(t),\ \ 0 \leqslant t \leqslant 0.1$<br/>
<p>Будем использовать различные разностные схемы: </p>
<p>1)  Явную схему порядка $\mathcal{O}(h^{2} + \tau)$ с аппроксимацией производных в граничных условияхс порядком $\mathcal{O}(h^{2})$</p>
<p>2)  Cхему с весами при $\sigma = 0, \sigma = 1, \sigma = 1/2$ с аппроксимацией производных в граничных условиях с порядком $\mathcal{O}(h)$</p>
<br/>
<p>Будем разбивать отрезок $[0, 1]$ на $ N $ равных частей. Обозначим $h = 1/N,\ \ x_{i} = ih,\ \ i = 0,\dots, N$</p>
<p>Будем разбивать отрезок $[0, 0.1]$ на $ M $ равных частей. Обозначим $\tau = 0.1/M,\ \ t_{k} = k\tau,\ \ k = 0,\dots M$</p>
<p>Будем строить сетку по узлам $(x_{i}, t_{k})$</p>
<p>Используя аппроксимации дифференциальных выражений разностными, заменяем оператор $Lu = \frac{\partial}{\partial x}((x+1)\frac{\partial u}{\partial x}) + 0\frac{\partial u}{\partial x} -u$ разностным оператором:</p>
$L_{h}u_{i}^{k} = (x_{i+\frac{1}{2}} +1)\frac{u_{i+1}^{k}- u_{i}^{k}}{h^{2}} - (x_{i-\frac{1}{2}} +1)\frac{u_{i}^{k}- u_{i-1}^{k}}{h^{2}} - u_{i}^{k}$

<h5>Явная схема</h5>
<p>Из начального условия имеем : $u_{i}^{0} = \varphi(x_{i}),\ \ i = 0,\dots, N$</p>
<p>Граничные условия аппроксимируем с порядком $\mathcal{O}(h^{2})$:</p>
$u_{0}^{k} = \alpha(t_{k})$<br/>
$u_{N}^{k} + \frac{3u_{N}^{k}-4u_{N-1}^{k}+u_{N-2}^{k}}{2h} = \beta(t_{k}), \ \ k = 1,\dots M$<br/>
<p>Аппроксимируем исходное уравнение:</p>
$\frac{u_{i}^{k} - u_{i}^{k-1}}{\tau} = L_{h}u_{i}^{k-1} + f(x_{i}, t_{k-1}),\ \ i = 1,\dots, N-1, \ \ k = 1,\dots M$<br/>
<p>Далее находим $u_{i}^{0}$, полагаем $k = 1$, находим $u_{i}^{k} = u_{i}^{k-1} + \tau(L_{h}u_{i}^{k-1} + f(x_{i}, t_{k-1}))$, находим $u_{0}^{k}$ и $u_{N}^{k}$ при $k = 1$. Далее повторяем данный алгоритм для всех $k \leqslant M$

In [10]:
# Подключение служебных пакетов
import sympy as sp
import math
import pandas as pd
import numpy as np

<h5>Реализация явного метода</h5>

In [30]:
def explicitSchema(alpha, beta, f, phi, N, M):
    x, t = sp.symbols("x t")
    u = np.zeros((M + 1, N + 1))
    h = 1/N
    tau = 0.1/M
    for i in range(0, N+1):
        u[0, i] = phi.subs(x, i*h)
    for k in range(1, M+1):
        t_k_1 = (k-1)*tau
        for i in range(1, N):
            x_i = i*h
            p_i_bef = (i-1/2)*h + 1
            p_i_aft = (i+1/2)*h + 1
            u[k, i] = u[k-1, i] + tau*(p_i_aft*(u[k-1, i+1] - u[k-1, i])/(h**2) - p_i_bef*(u[k-1, i] - u[k-1, i-1])/(h**2)- u[k-1, i] +
                                      f.subs([(x, x_i), (t, t_k_1)]))
        t_k = k*tau
        u[k, 0] = alpha.subs(t, t_k)
        u[k, N] = (beta.subs(t, t_k) - (-4*u[k, N-1]+u[k, N-2])/(2*h))/(1+3/(2*h))
    
    return u[::int(M/5), ::int(N/5)]

<h5>Неявная схема</h5>
<p>Из начального условия имеем : $u_{i}^{0} = \varphi(x_{i}),\ \ i = 0,\dots, N$</p>
<p>Граничные условия аппроксимируем с порядком $\mathcal{O}(h)$:</p>
$u_{0}^{k} = \alpha(t_{k})$<br/>
$u_{N}^{k} + \frac{u_{N}^{k}-u_{N-1}^{k}}{h} = \beta(t_{k}), \ \ k = 1,\dots M$<br/>
<p>Пусть $\sigma$ — вещественный параметр. Рассмотрим однопараметрическое семейство разностных схем:</p>
$\frac{u_{i}^{k} - u_{i}^{k-1}}{\tau} = L_{h}(\sigma u_{i}^{k} + (1-\sigma)u_{i}^{k-1}) + f(x_{i}, \overline{t_{k}}),\ \ i = 1,\dots, N-1, \ \ k = 1,\dots M$<br/>
<p>Так как к моменту определения решения на k-ом слое решение на предыдущем (k−1)-ом слое уже известно, поэтому последнюю систему перепишем следующим образом:</p>
$\sigma L_{h}u_{i}^{k} - \frac{1}{\tau}u_{i}^{k} = G_{i}^{k}$
<p>Таким образом, на каждом k-ом слое в данном случае приходится решать систему (N+1) порядка с трехдиагональной матрицей c коэффициентами:</p>
$A_{i} = \frac{\sigma (x_{i-\frac{1}{2}} +1)}{h^{2}}$<br/>
$B_{i} = \frac{\sigma \tau(x_{i-\frac{1}{2}} + x_{i+\frac{1}{2}}+ 2 + h^{2})+h^{2}}{h^{2} \tau}$<br/>
$C_{i} = \frac{\sigma (x_{i+\frac{1}{2}} +1)}{h^{2}}$

<h5>Реализация неявного метода</h5>

In [33]:
def implicitSchema(alpha, beta, f, phi, N, M, sigma):
    x, t = sp.symbols("x t")
    A = np.zeros(N+1)
    B = np.zeros(N+1)
    C = np.zeros(N+1)
    G = np.zeros(N+1)
    u = np.zeros((M + 1, N + 1))
    h = 1/N
    tau = 0.1/M
    for i in range(0, N+1):
        u[0, i] = phi.subs(x, i*h)
    param = (sigma-1)*tau
    for k in range(1, M+1):
        t_Line = k*tau
        t_k = t_Line + param
        for i in range(1, N):
            x_i = i*h
            p_i_bef = (i-1/2)*h + 1
            p_i_aft = (i+1/2)*h + 1
            A[i] = sigma*p_i_bef/(h**2)
            B[i] = (sigma*tau*(p_i_bef+p_i_aft + h**2)+h**2)/(tau*h**2)
            C[i] = sigma*p_i_aft/(h**2)
            G[i] = -(1/tau)*u[k-1, i]-(1-sigma)*(p_i_aft*(u[k-1, i+1] - u[k-1, i])/(h**2) - p_i_bef*(u[k-1, i] - u[k-1, i-1])/(h**2)- u[k-1, i])- f.subs([(x, x_i), (t, t_k)])
        A[0] = 0
        A[N] = -1/h
        B[0] = -1
        B[N] = -(1+1/h)
        C[0] = 0
        C[N] = 0
        G[0] = alpha.subs(t, t_Line)
        G[N] = beta.subs(t, t_Line)
        res = solveBanded(A, B, C, G, N+1)
        
        for i in range(0, N+1):
            u[k,i] = res[i]
    return u[::int(M/5), ::int(N/5)]

<h5>Тестирование алгоритма на решениях, для которых разностная схема точно аппроксимирует дифференциальную задачу</h5>
<p>Пускай таким решением будет $u(x,t) = C$, где C - константа. Возьмем С = 1</p>
<p>Тогда из исходного уравнения получаем, что $f(x,t) \equiv 1$</p>
<p>А также, $\varphi(x) \equiv 1$, $\alpha(x) \equiv 1$, $\beta(x) \equiv 1$</p>


In [63]:
x, t = sp.symbols("x, t")

u_C = sp.simplify(1)
alpha_C = sp.simplify(1)
beta_C = sp.simplify(1)
f_C = sp.simplify(1)
phi_C = sp.simplify(1)

# Построим таблицу с точным решением
solution = generateTable(u_C)
table_C = bigTable(solution)
print(table_C)

    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0  1.0
5  0.10  1.0  1.0  1.0  1.0  1.0  1.0


<h5>Проверим явную схему</h5>

In [64]:
start(solution, alpha_C, beta_C, f_C, phi_C)

Таблицы решения на “крупной” сетке независимо от шагов по t и x, с которыми строится решение
--------------------------------------------------------------------------------------------
h = 0.2
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0  1.0
5  0.10  1.0  1.0  1.0  1.0  1.0  1.0

h = 0.1
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0  1.0
5  0.10  1.0  1.0  1.0  1.0  1.0  1.0

h = 0.05
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0  1.0
5  0.10  1.0  1.0  1.0  1.0

<h5>Проверим неявную схему</h5>

In [65]:
start(solution, alpha_C, beta_C, f_C, phi_C, 0)

Таблицы решения на “крупной” сетке независимо от шагов по t и x, с которыми строится решение
--------------------------------------------------------------------------------------------
h = 0.2, sigma = 0
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0  1.0
5  0.10  1.0  1.0  1.0  1.0  1.0  1.0

h = 0.1, sigma = 0
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0  1.0
5  0.10  1.0  1.0  1.0  1.0  1.0  1.0

h = 0.05, sigma = 0
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0

In [66]:
start(solution, alpha_C, beta_C, f_C, phi_C, 0.5)

Таблицы решения на “крупной” сетке независимо от шагов по t и x, с которыми строится решение
--------------------------------------------------------------------------------------------
h = 0.2, sigma = 0.5
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0  1.0
5  0.10  1.0  1.0  1.0  1.0  1.0  1.0

h = 0.1, sigma = 0.5
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0  1.0
5  0.10  1.0  1.0  1.0  1.0  1.0  1.0

h = 0.05, sigma = 0.5
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.

In [67]:
start(solution, alpha_C, beta_C, f_C, phi_C, 1)

Таблицы решения на “крупной” сетке независимо от шагов по t и x, с которыми строится решение
--------------------------------------------------------------------------------------------
h = 0.2, sigma = 1
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0  1.0
5  0.10  1.0  1.0  1.0  1.0  1.0  1.0

h = 0.1, sigma = 1
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0  1.0
5  0.10  1.0  1.0  1.0  1.0  1.0  1.0

h = 0.05, sigma = 1
    t/x    0  0.2  0.4  0.6  0.8    1
0  0.00  1.0  1.0  1.0  1.0  1.0  1.0
1  0.02  1.0  1.0  1.0  1.0  1.0  1.0
2  0.04  1.0  1.0  1.0  1.0  1.0  1.0
3  0.06  1.0  1.0  1.0  1.0  1.0  1.0
4  0.08  1.0  1.0  1.0  1.0  1.0

<h5>Тестирование алгоритма на решении $u(x, t) = x^{3}+t^{3}$</h5>
<p>В этом случае получим, что </p>
$f(x,t) = t^{3} + t^{2} + x^{3} -9x^{2} - 6x$<br/>
$\varphi(x) = x^{3}$<br/>
$\alpha(t) = t^{3}$<br/>
$\beta(t) = t^{3} + 4$

In [68]:
x, t = sp.symbols("x, t")

u_C = sp.simplify(x**3 + t**3)
alpha_C = sp.simplify(t**3)
beta_C = sp.simplify(t**3 + 4)
f_C = sp.simplify(t**3 + t**2 + x**3 - 9*x**2 - 6*x)
phi_C = sp.simplify(x**3)

# Построим таблицу с точным решением
solution = generateTable(u_C)
table_C = bigTable(solution)
print(table_C)

    t/x         0       0.2       0.4       0.6       0.8         1
0  0.00  0.000000  0.008000  0.064000  0.216000  0.512000  1.000000
1  0.02  0.000008  0.008008  0.064008  0.216008  0.512008  1.000008
2  0.04  0.000064  0.008064  0.064064  0.216064  0.512064  1.000064
3  0.06  0.000216  0.008216  0.064216  0.216216  0.512216  1.000216
4  0.08  0.000512  0.008512  0.064512  0.216512  0.512512  1.000512
5  0.10  0.001000  0.009000  0.065000  0.217000  0.513000  1.001000


<h5>Проверим явную схему</h5>

In [69]:
start(solution, alpha_C, beta_C, f_C, phi_C)

Таблицы решения на “крупной” сетке независимо от шагов по t и x, с которыми строится решение
--------------------------------------------------------------------------------------------
h = 0.2
    t/x         0       0.2       0.4       0.6       0.8         1
0  0.00  0.000000  0.008000  0.064000  0.216000  0.512000  1.000000
1  0.02  0.000008  0.008687  0.064797  0.216797  0.517245  1.015349
2  0.04  0.000064  0.009157  0.066205  0.221044  0.523280  1.021206
3  0.06  0.000216  0.010166  0.068534  0.224802  0.527755  1.025384
4  0.08  0.000512  0.011516  0.070977  0.228111  0.531394  1.028727
5  0.10  0.001000  0.013003  0.073351  0.231072  0.534518  1.031588

h = 0.1
    t/x         0       0.2       0.4       0.6       0.8         1
0  0.00  0.000000  0.008000  0.064000  0.216000  0.512000  1.000000
1  0.02  0.000008  0.008170  0.064217  0.216464  0.513531  1.003989
2  0.04  0.000064  0.008358  0.064675  0.217425  0.514971  1.005446
3  0.06  0.000216  0.008683  0.065292  0.218365  

<h5>Проверим неявную схему</h5>

In [70]:
start(solution, alpha_C, beta_C, f_C, phi_C, 0)

Таблицы решения на “крупной” сетке независимо от шагов по t и x, с которыми строится решение
--------------------------------------------------------------------------------------------
h = 0.2, sigma = 0
    t/x         0       0.2       0.4       0.6       0.8         1
0  0.00  0.000000  0.008000  0.064000  0.216000  0.512000  1.000000
1  0.02  0.000008  0.008687  0.064797  0.216797  0.557099  1.130917
2  0.04  0.000064  0.009157  0.072556  0.249117  0.597798  1.164843
3  0.06  0.000216  0.014991  0.087562  0.273842  0.625315  1.187798
4  0.08  0.000512  0.022942  0.102705  0.294045  0.646341  1.205370
5  0.10  0.001000  0.030996  0.116546  0.311076  0.663425  1.219688

h = 0.1, sigma = 0
    t/x         0       0.2       0.4       0.6       0.8         1
0  0.00  0.000000  0.008000  0.064000  0.216000  0.512000  1.000000
1  0.02  0.000008  0.008170  0.064571  0.221079  0.535442  1.063335
2  0.04  0.000064  0.009218  0.069413  0.233585  0.554397  1.080994
3  0.06  0.000216  0.012037

In [71]:
start(solution, alpha_C, beta_C, f_C, phi_C, 0.5)

Таблицы решения на “крупной” сетке независимо от шагов по t и x, с которыми строится решение
--------------------------------------------------------------------------------------------
h = 0.2, sigma = 0.5
    t/x         0       0.2       0.4       0.6       0.8         1
0  0.00  0.000000  0.008000  0.064000  0.216000  0.512000  1.000000
1  0.02  0.000008  0.008995  0.066991  0.228369  0.561144  1.134288
2  0.04  0.000064  0.012005  0.077189  0.253703  0.600146  1.166799
3  0.06  0.000216  0.017722  0.090970  0.276382  0.626740  1.188986
4  0.08  0.000512  0.024779  0.104864  0.295557  0.647202  1.206087
5  0.10  0.001000  0.032047  0.117742  0.311877  0.663860  1.220050

h = 0.1, sigma = 0.5
    t/x         0       0.2       0.4       0.6       0.8         1
0  0.00  0.000000  0.008000  0.064000  0.216000  0.512000  1.000000
1  0.02  0.000008  0.008245  0.065066  0.222066  0.536203  1.063834
2  0.04  0.000064  0.009549  0.069997  0.234144  0.554745  1.081242
3  0.06  0.000216  0.01

In [72]:
start(solution, alpha_C, beta_C, f_C, phi_C, 1)

Таблицы решения на “крупной” сетке независимо от шагов по t и x, с которыми строится решение
--------------------------------------------------------------------------------------------
h = 0.2, sigma = 1
    t/x         0       0.2       0.4       0.6       0.8         1
0  0.00  0.000000  0.008000  0.064000  0.216000  0.512000  1.000000
1  0.02  0.000008  0.009897  0.070044  0.235106  0.567930  1.139943
2  0.04  0.000064  0.014040  0.081195  0.258145  0.602997  1.169175
3  0.06  0.000216  0.019857  0.094131  0.279049  0.628309  1.190293
4  0.08  0.000512  0.026438  0.107000  0.297143  0.648094  1.206831
5  0.10  0.001000  0.033136  0.119028  0.312739  0.664296  1.220413

h = 0.1, sigma = 1
    t/x         0       0.2       0.4       0.6       0.8         1
0  0.00  0.000000  0.008000  0.064000  0.216000  0.512000  1.000000
1  0.02  0.000008  0.008349  0.065517  0.222980  0.536977  1.064363
2  0.04  0.000064  0.009853  0.070555  0.234708  0.555103  1.081496
3  0.06  0.000216  0.012674

<h5>Функция для запуска решения явным и неявным методами</h5>

In [28]:
def start(u_C, alpha, beta, f, phi, sigma = -1):
    N = [5, 10, 20]
    M = [10, 40, 160]
        
    sol_error = np.zeros(3)
    sol_norma = np.zeros(3)
    
    u = np.zeros((6, 6))
    u_pred = np.zeros((6, 6))
    
    print("Таблицы решения на “крупной” сетке независимо от шагов по t и x, с которыми строится решение")
    print("--------------------------------------------------------------------------------------------")
    
    for i in range(3):
        h = 1/N[i]
        u_pred = u.copy()
        if(sigma == -1):
            u = explicitSchema(alpha, beta, f, phi, N[i], M[i])
        else:
            u = implicitSchema(alpha, beta, f, phi, N[i], M[i], sigma)
            
        if(sigma == -1):
            print(f"h = {h}")
        else:
            print(f"h = {h}, sigma = {sigma}")
            
        print(bigTable(u))
        
        sol_error[i] = norma(u_C, u)
        if i > 0:
            sol_norma[i] = norma(u, u_pred)
        print()
        
    print()
    print("Таблица, характеризующие точность решения и внутреннюю сходимость")
    print("-----------------------------------------------------------------") 
    df = pd.DataFrame({ 
                   'h': [0.2, 0.1, 0.05],
                   'tau': [0.1/M[0], 0.1/M[1], 0.1/M[2]], 
                   '||J_ex - u||': sol_error[:],
                   '||u^h - u^2h||': ["----", sol_norma[1], sol_norma[2]]
                  })
    print(df)
    print("-----------------------------------------------------------------")
    

<h5>Функция для поиска нормы</h5>

In [24]:
def norma(u_Ex, u_Sol):
    u = u_Ex - u_Sol;
    maxV = -np.inf
    for i in range(6):
        for j in range(6):
            if abs(u[i,j]) > maxV:
                maxV = abs(u[i,j])
    return maxV

<h5>Функция для генерации таблицы точного решения на “крупной” сетке</h5>

In [59]:
def generateTable(u):
    x, t = sp.symbols("x t")
    res = np.zeros((6, 6))
    for j in range(6):
        t_j = 0.02*j 
        for i in range(6):
            x_i = 0.2*i
            res[j, i] = u.subs([(x, x_i), (t, t_j)])
    return res

<h5>Функция для печати таблицы решения на “крупной” сетке</h5>

In [13]:
def bigTable(u):
    
    df = pd.DataFrame({ 
                   't/x': [0, 0.02, 0.04, 0.06, 0.08, 0.1],
                   '0': u[:, 0], 
                   '0.2': u[:, 1],
                   '0.4': u[:, 2],
                   '0.6': u[:, 3],
                   '0.8': u[:, 4],
                   '1': u[:, 5] 
                  })
    
    return df

<h5>Функция метода прогонки</h5>

In [35]:
def solveBanded(A, B, C, G, n):
    res = np.zeros(n)
    s = np.zeros(n)
    t = np.zeros(n)
    
    s[0] = C[0]/B[0]
    t[0] = -G[0]/B[0]
    
    for i in range(1, n):
        s[i] = C[i]/(B[i]-A[i]*s[i-1])
        t[i] = (A[i]*t[i-1] - G[i])/(B[i]-A[i]*s[i-1])        
    res[n-1] = t[n-1]
    for i in range(-2, -1*(n+1), -1):
        res[i] = s[i]*res[i+1] + t[i]
    return res